In [95]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [96]:
# Prompting with openai
def get_completion_and_token_count(
    prompt, model="gpt-3.5-turbo", temperature=0, max_tokens=3000
):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "completion_tokens": response["usage"]["completion_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
    }

    return content, token_dict

"""For this task, I want you to act as a text prompt generator for creating image with AI. "
    "Your main job is to express <explanation_of_expression> with emphasis on <Expression>."
    "\n\nInstruction 1 - provide at least 3 sentences long, detailed image description"
    "\nInstruction 2 - in the description, you must not use any of the following objects:"
    "\npeople; person; persona; professional; human; hands; face; poster; signs; letters; text"
    "\n\nAfter the description is created, check whether the Instruction 2 was fulfilled. "
    "If not, create another image description according to all Instructions."
"""
prompt = """
I want to create additional information of words in my vocabulary list.

additional_translations:
 - Find translations different from the ones that were given, but with the same meaning

explanation_of_expression:
 - Give explanation for the expression with English text in one sentence

explanation_of_syllables:
 - For each syllable in the Expression, find its independent meaning, if it has one, use one sentence to explain it

image_description:
 - Create 3 sentence long Image Description according to the explanation_of_expression that will be easily expressed by an image
 - Provide it with many detailed objects, with extra emphasis on the Expression

image_tags:
 - Create ten tags that describe the background details that are closely related with the translations and explanation_of_expression

sentence_example:
 - Create a short sentence example of usage according to the explanation_of_expression with translation

Do it for these words:
Expression: 비빔밥, Translation: Bibimbap
"""

In [ ]:
response, token_dict = get_completion_and_token_count(prompt)
print(response)
print(token_dict)

In [ ]:
# Prompting with LangChain - Outputing Dictionaries/json
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback

llm = ChatOpenAI(temperature=0.0, verbose=True)


In [ ]:
schema_queries = {
    "additional_translations": (
        "Create additional English translations that are same in meaning but are different from "
        "<Expression>."
    ),
    "explanation_of_expression": (
        "Create 1 sentence long explanation for "
        "<Expression>."
    ),
    "explanation_of_syllables": (
        "Explain the meaning of each syllable in "
        "<Expression>. Explain it only if it can be used as a Korean word, otherwise leave it blank."
    ),
    "image_description": (
        "Create 3 sentence long Image Description according to the "
        "<explanation_of_expression> that will be easily expressed by an image. "
        "Provide it with many detailed objects, with extra emphasis on the "
        "<Expression>."
    ),
    "image_tags": (
        "Create 10 tags that describe the background details that are closely related with: "
        "<explanation_of_expression>. Use adjactives only!"
    ),
    "sentence_example": (
        "Show how the "
        "<Expression> can be used in a simple korean sentence. "
        "Add translation into brackets."
    ),
}

response_schemas = []
queries = ""
for schema, query in schema_queries.items():
    response_schemas.append(ResponseSchema(name=schema, description=""))
    queries += f"<{schema}>:\n{query}\n\n"

review_template = f"""
I want to create additional information of words in my vocabulary list.

{queries}""" + """
Do it for this word:
<Expression>: {korean}, <Translation>: {translation}

{format_instructions}
"""

review_template

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(korean="거울", translation="mirror", format_instructions=format_instructions)


In [ ]:
# for Tokens Used printing
# with get_openai_callback() as cb:
#     print(cb)
response = llm(messages)
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [ ]:
chain_inputs = [
    ["korean"],
    ["korean"],
    ["korean"],
    ["explanation_of_expression", "korean"],
    ["explanation_of_expression", "translation"],
    ["explanation_of_expression"],
]
chain_labels = {
    "additional_translations": (
        "Find other additional translations that are same in meaning but are different from this expression:"
        "\n{korean}"
    ),
    "explanation_of_expression": (
        "Give one sentence explanation using only English characters for this expression:"
        "\n{korean}"
    ),
    "explanation_of_syllables": (
        "Explain every syllable (if it has meaning) in one sentence for this expression:"
        "\n{korean}"
    ),
    "image_description": (
        "Create 3 sentence long image description that would be easily expressed by an image according to this explanation:"
        "\n{explanation_of_expression}"
        "\n\nProvide it with many detailed objects, with extra emphasis on this expression:"
        "\n{korean}"
    ),
    "image_tags": (
        "Create ten tags that describe the background details that are closely related with:"
        "\nthis translations - {translation}"
        "\nthis explanation - {explanation_of_expression}"
    ),
    "sentence_example": (
        "Create a short sentence example of usage with translation according to this explanation:"
        "\n{explanation_of_expression}"
    ),
}
chains = []
outputs = []
for chain_label, input_vars in zip(chain_labels, chain_inputs):
    prompt = ChatPromptTemplate.from_template(
        template=chain_labels[chain_label]
    )
    chain = LLMChain(llm=llm, prompt=prompt, output_key=chain_label)
    chains.append(chain)
    outputs.append(chain_label)

overall_chain = SequentialChain(
    chains=chains,
    input_variables=["korean", "translation"],
    output_variables=outputs,
    verbose=True,
)


In [ ]:
overall_chain({"korean":"감사하다", "translation": "appreciate, be thankful"})